In [ ]:
import pandas as pd
import requests
from datetime import datetime

In [ ]:
def fetch_crypto_data(crypto_id, start_date):
    # Calculate the number of days from the start date to today
    start_date = pd.to_datetime(start_date)
    today = pd.to_datetime(datetime.now())
    days = (today - start_date).days

    if days < 1:
        return None

    # Limit the number of days to a maximum of 365
    days = min(days, 365)

    url = 'enter your api key (not shared due to limited access)'
    params = {
        'vs_currency': 'usd',
        'days': days,          # Set the number of days
        'interval': 'daily'    # Daily interval
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        prices = data['prices']

        # Creating DataFrame from the prices
        df = pd.DataFrame(prices, columns=['timestamp', 'price'])
        df['Date'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('Date', inplace=True)

        df['Open'] = df['price'].shift(1)  # Previous day's price as Open
        df['Close'] = df['price']
        df['High'] = df['price'].rolling(window=2).max()  # Max of last 2 prices
        df['Low'] = df['price'].rolling(window=2).min()   # Min of last 2 prices

        df = df[['Open', 'High', 'Low', 'Close']]

        return df
    else:
        print(f"Error: Unable to fetch data, received status code {response.status_code}")
        return None

In [ ]:
crypto_id = 'bitcoin'
start_date = '2023-01-01'
df_crypto = fetch_crypto_data(crypto_id, start_date)

In [ ]:
df_crypto.to_csv('bitcoin.csv')

In [ ]:
df = pd.read_csv('bitcoin.csv')

In [ ]:
df